In [2]:
import numpy as np
from xgbsurv.models.utils import transform, transform_back
from xgbsurv.models.eh_final import eh_likelihood, eh_gradient
from xgbsurv.models.eh_ah_final import ah_likelihood, ah_objective
from xgbsurv.models.eh_aft_final import aft_likelihood, aft_objective
import sys
sys.path.append('/Users/JUSC/Documents/xgbsurv_benchmarking/deep_learning/')
from loss_functions_pytorch import eh_likelihood_torch, eh_likelihood_torch_2, aft_likelihood_torch, ah_likelihood_torch
import torch
import math
torch.set_printoptions(precision=10)
from torch.autograd.functional import hessian

In [3]:
# create data function

def eh_data(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]]) #.reshape(1,10)
        y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32) #.reshape(1,10)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]], requires_grad=True)#.reshape(10,1)
        y = torch.tensor([[1, -3, -3, -4, -7,  8,  9,  -11,  13,  16]], dtype=torch.float32) #.reshape(10,1)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event

def eh_data_2(type='np'):
    #h2==0 scenario
    if type=='np':
        linear_predictor = np.array([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]]) #.reshape(1,10)
        # attention reshape changes order in certain way
        y = np.array([[1, 1],[ -3, -3],[-3,-3],[ -4,-4],[ -7,-7],[ 8,8],[ 9,9],[ -11,-11],[ 13,13],[ 16,16]],dtype=np.float32)
        #y = np.array([1, -3, -3, -4, -7,  8,  9,  -11,  13,  16],dtype=np.float32) #.reshape(1,10)
        time = np.array([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = np.array([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=np.float32)
    if type=='torch':
        linear_predictor = torch.tensor([[0.67254923, 0.03356795],
       [0.86077982, 0.65922692],
       [0.43557393, 0.75447972],
       [0.94059047, 0.30572004],
       [0.8446509 , 0.07916267],
       [0.23657039, 0.44693716],
       [0.74629685, 0.32637245],
       [0.99700768, 0.10225456],
       [0.28182768, 0.05405025],
       [0.44495038, 0.08454563]], requires_grad=True)#.reshape(10,1)
        y = torch.tensor([[1, 1],[ -3, -3],[-3,-3],[ -4,-4],[ -7,-7],[ 8,8],[ 9,9],[ -11,-11],[ 13,13],[ 16,16]],dtype=torch.float32).view(10,2)
        time = torch.tensor([[ 1,  3,  3,  4,  7,  8,  9, 11, 13, 16]])
        event = torch.tensor([[1, 0, 0, 0, 0, 1, 1, 0, 1, 1]],dtype=torch.float32).reshape(10,1)

    return y, linear_predictor, time, event



In [4]:
y, linear_predictor, time, event = eh_data(type='torch')

## Structure

- Compare loss to original function

## 1. Compare loss to original function



In [5]:
def transform_back_torch(y: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
    """Transforms XGBoost digestable format variable y into time and event.

    Parameters
    ----------
    y : npt.NDArray[float]
        Array containing survival time and event where negative value is taken as censored event.

    Returns
    -------
    tuple[npt.NDArray[float],npt.NDArray[int]]
        Survival time and event.
    """
    time = torch.abs(y)
    event = (torch.abs(y) == y)
    event = event # for numba
    return time, event.float()

def eaftloss(out, y): ##loss function for AFT or EH
    time, delta = transform_back_torch(y)
    ia, ib = out.size()
    if ib == 1: ###loss function for AFT
        n = len(delta)
        print('aft')
        h = 1.30*math.pow(n,-0.2)
        #h 1.304058*math.pow(n,-0.2)  ## 1.304058*n^(-1/5) or 1.587401*math.pow(n,-0.333333) 1.587401*n^(-1/3)
        time = time.view(n,1)
        delta = delta.view(n,1)
        
        # R = g(Xi) + log(Oi)
        R = torch.add(out,torch.log(time)) 
        
        # Rj - Ri
        rawones = torch.ones([1,n], dtype = out.dtype)
        R1 = torch.mm(R,rawones)
        R2 = torch.mm(torch.t(rawones),torch.t(R))
        DR = R1 - R2 
        
        # K[(Rj-Ri)/h]
        K = normal_density(DR/h)
        Del = torch.mm(delta, rawones)
        DelK = Del*K 
        
        # (1/nh) *sum_j Deltaj * K[(Rj-Ri)/h]
        Dk = torch.sum(DelK, dim=0)/(n*h)
        
        # log {(1/nh) * Deltaj * K[(Rj-Ri)/h]}    
        log_Dk = torch.log(Dk)     
        A = torch.t(delta)*log_Dk/n   
        S1 = A.sum()  
        
        ncdf=torch.distributions.normal.Normal(torch.tensor([0.0], dtype = out.dtype), torch.tensor([1.0], dtype = out.dtype)).cdf
        P = ncdf(DR/h)
        CDF_sum = torch.sum(P, dim=0)/n
        Q = torch.log(CDF_sum)
        S2 = -(delta*Q.view(n,1)).sum()/n
             
        S0 = -(delta*torch.log(time)).sum()/n
        
        S = S0 + S1 + S2 
        S = -S
    else: ### loss function for Extended hazard model
        print('eh model')
        print(y)
        n = len(out[:,0])
        h = 1.30*math.pow(n,-0.2)  ## or 1.59*n^(-1/3)
        print('bandwidth', h)
        time = time.view(n,1)
        delta = delta.view(n,1)
        g1 = out[:,0].view(n,1)
        g2 = out[:,1].view(n,1)
        
        # R = g(Xi) + log(Oi)
        R = torch.add(g1,torch.log(time)) 
        
        S1 =  (delta*g2).sum()/n
        S2 = -(delta*R).sum()/n
        print('S1,S2', S1, S2)
        
        # Rj - Ri
        rawones = torch.ones(1,n)
        R1 = torch.mm(R,rawones)
        R2 = torch.mm(torch.t(rawones),torch.t(R))
        DR = R1 - R2 
        
        # K[(Rj-Ri)/h]
        K = normal_density(DR/h)
        Del = torch.mm(delta, rawones)
        DelK = Del*K 
        
        # (1/nh) *sum_j Deltaj * K[(Rj-Ri)/h]
        Dk = torch.sum(DelK, dim=0)/(n*h)  ## Dk would be zero as learning rate too large!
        
        # log {(1/nh) * Deltaj * K[(Rj-Ri)/h]}    
        log_Dk = torch.log(Dk)    
        
        S3 = (torch.t(delta)*log_Dk).sum()/n    
        
        # Phi((Rj-Ri)/h)
        ncdf=torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0])).cdf
        P = ncdf(DR/h) 
        L = torch.exp(g2-g1)
        LL = torch.mm(L,rawones)
        LP_sum = torch.sum(LL*P, dim=0)/n
        Q = torch.log(LP_sum)
        
        S4 = -(delta*Q.view(n,1)).sum()/n
        print(S1+ S2, S3, S4)
        S = S1 + S2 + S3 + S4  
        S = -S
    return S

def normal_density(a):  
    b = 0.3989423*torch.exp(-0.5*torch.pow(a,2.0))
    return b

### EH Loss Original Paper

In [6]:
y, linear_predictor, time, event = eh_data(type='torch')
aft_loss_paper = eaftloss(linear_predictor, y)
print(aft_loss_paper ) #-1.1941503219595515

eh model
tensor([[  1.,  -3.,  -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.]])
bandwidth 0.8202445478242513
S1,S2 tensor(0.0945473462, grad_fn=<DivBackward0>) tensor(-1.1996399164, grad_fn=<DivBackward0>)
tensor(-1.1050925255, grad_fn=<AddBackward0>) tensor(-0.9911370277, grad_fn=<DivBackward0>) tensor(0.6148166060, grad_fn=<DivBackward0>)
tensor(1.4814128876, grad_fn=<NegBackward0>)


### EH Loss Pytorch Torch Version

check bandwidth, check dims vector for aft, go through code step by step

In [7]:
y, linear_predictor, time, event = eh_data_2(type='torch')
print(linear_predictor.shape,y.shape)
eh_loss_own_torch = eh_likelihood_torch_2(linear_predictor,y)
eh_loss_own_torch

torch.Size([10, 2]) torch.Size([10, 2])
bandwidth tensor(0.8202445507)
tensor(0.0945473462, grad_fn=<DivBackward0>) tensor(1.1996399164, grad_fn=<DivBackward0>) tensor(-0.9911370277, grad_fn=<DivBackward0>) tensor(-0.6148166656, grad_fn=<DivBackward0>)


tensor(1.4814128876, grad_fn=<NegBackward0>)

## EH Loss Numba

In [8]:
y, linear_predictor, time, event = eh_data_2(type='np')
print(linear_predictor.shape,y.shape)
eh_loss_own_numba = eh_likelihood(
     y, linear_predictor
)
eh_loss_own_numba

(10, 2) (10, 2)


1.48141275238739

### EH Loss Pytorch Numba Version - WRONG

In [9]:
# y, linear_predictor, time, event = eh_data_2(type='torch')
# print(linear_predictor.shape,y.shape)
# eh_loss_own_torch_numba = eh_likelihood_torch(linear_predictor,y)
# eh_loss_own_torch_numba

### Loss Function Comparison

In [10]:
np.allclose(eh_loss_own_numba ,eh_loss_own_torch.detach().numpy())

True

## Gradient Comparison

## EH Grad Paper

In [11]:
y, linear_predictor, time, event = eh_data(type='torch')
eh_loss_paper = eaftloss(linear_predictor, y)
eh_loss_paper.backward()
eh_grad_paper = linear_predictor.grad
eh_grad_paper

eh model
tensor([[  1.,  -3.,  -3.,  -4.,  -7.,   8.,   9., -11.,  13.,  16.]])
bandwidth 0.8202445478242513
S1,S2 tensor(0.0945473462, grad_fn=<DivBackward0>) tensor(-1.1996399164, grad_fn=<DivBackward0>)
tensor(-1.1050925255, grad_fn=<AddBackward0>) tensor(-0.9911370277, grad_fn=<DivBackward0>) tensor(0.6148166060, grad_fn=<DivBackward0>)


tensor([[ 0.0281834509, -0.0957000479],
        [ 0.0066343807,  0.0305879973],
        [ 0.0060651265,  0.0306768492],
        [ 0.0036482103,  0.0305661000],
        [-0.0052330643,  0.0427651592],
        [-0.0456178114, -0.0296885744],
        [ 0.0081371665, -0.0313907117],
        [-0.0278914571,  0.0571271330],
        [-0.0044439435, -0.0231527984],
        [ 0.0305179432, -0.0117911026]])

array([-0.09570005,  0.030588  ,  0.03067684,  0.0305661 ,  0.04276516,
       -0.02968857, -0.03139072,  0.05712713, -0.0231528 , -0.01179111])

### Gradient Own Pytorch



In [12]:
# y, linear_predictor, time, event = eh_data_2(type='torch')
# #linear_predictor, y  = linear_predictor.reshape(-1), y.reshape(-1)
# ah_loss_own_torch = eh_likelihood_torch(linear_predictor,y)
# print('loss', eh_loss_own_torch)
# eh_loss_own_torch.backward()
# print(linear_predictor.is_leaf)
# #linear_predictor.retain_grad()
# grad_torch = linear_predictor.grad
# print('grad_torch', grad_torch.numpy())

## Gradient Numba

In [16]:
y, linear_predictor, time, event = eh_data_2(type='np')
grad_own = eh_gradient(
    y, linear_predictor
)
grad_own

array([[ 0.02818345, -0.09570005],
       [ 0.00663439,  0.030588  ],
       [ 0.00606514,  0.03067684],
       [ 0.00364822,  0.0305661 ],
       [-0.00523305,  0.04276516],
       [-0.04561782, -0.02968857],
       [ 0.00813714, -0.03139072],
       [-0.02789145,  0.05712713],
       [-0.00444395, -0.0231528 ],
       [ 0.03051793, -0.01179111]])

### Gradient Function Comparison

In [17]:
np.allclose(grad_own, eh_grad_paper)

True

## Hessian Function Comparison

In [ ]:

from torch.autograd.functional import hessian as hess_torch
y, linear_predictor, time, event = ah_data_2(type='torch')
print(y.shape, linear_predictor.shape)
hessian_matrix = hess_torch(ah_likelihood_torch, (linear_predictor, y), create_graph=True)
diag_hessian = hessian_matrix[0][0].diag()
diag_hessian


NameError: name 'ah_data_2' is not defined

In [ ]:
y, linear_predictor, time, event = ah_data_2(type='np')
grad_own, hess_own = ah_objective(
    y, linear_predictor
)
hess_own

array([0.1671356 , 0.09057899, 0.08653509, 0.10661001, 0.1241162 ,
       0.18680498, 0.242457  , 0.08737591, 0.30874032, 0.27100193])

In [ ]:
diag_hessian

tensor([2.])